In [31]:
!pip install tqdm
!pip install torch-summary
!pip install -U scikit-learn
!pip install tensorboard

In [32]:
SHOULD_PRINT = True
SEED = 32
CONTINUE_MODEL = None

In [33]:
import torch
from tqdm import tqdm

torch.manual_seed(SEED)

In [34]:
DEVICE = "cpu"
if torch.cuda.is_available():
    DEVICE = "cuda"

# Dataset

In [35]:
import torch
import numpy as np
from typing import List
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import random

class CustomDataset(Dataset):
    def __init__(self, large_file_path, chunk_size, subset_size=50000):
        self.large_file_path = large_file_path
        self.chunk_size = chunk_size
        self.subset_size = subset_size
        self.line_offsets = self.get_line_offsets(large_file_path, chunk_size)
        self.scaler = StandardScaler()
        print("Calculating mean and std...")
        self.mean, self.std = self.calculate_mean_std()
        print(f"Mean: {self.mean}, Std: {self.std}")

    def get_line_offsets(self, path: str, chunk_size: int) -> List[int]:
        offsets = [0]
        with open(path, "rb") as file:
            chunk = file.readlines(chunk_size)
            while chunk:
                for line in chunk:
                    offsets.append(offsets[-1] + len(line))
                chunk = file.readlines(chunk_size)
                print(f"Lines found: {len(offsets)}", end='\r')
        offsets = offsets[:-1]
        print(f"Lines found: {len(offsets)}", end='\n')
        return offsets

    def calculate_mean_std(self):
        selected_offsets = random.sample(self.line_offsets, min(self.subset_size, len(self.line_offsets)))
        features = []
        for offset in selected_offsets:
            with open(self.large_file_path, 'r', encoding='utf-8') as f:
                f.seek(offset)
                line = f.readline()
                numbers = [float(num) for num in line.strip().split()]
                features.append(numbers[:4])
        features = np.array(features)
        mean = np.mean(features, axis=0, dtype=np.float32)
        std = np.std(features, axis=0, dtype=np.float32)
        return mean, std

    def standardize_features(self, features):
        standardized_features = (features - self.mean) / self.std
        return standardized_features

    def __len__(self):
        return len(self.line_offsets)

    def __getitem__(self, line):
        offset = self.line_offsets[line]
        with open(self.large_file_path, 'r', encoding='utf-8') as f:
            f.seek(offset)
            line = f.readline()
            numbers = [float(num) for num in line.strip().split()]
            features, targets = numbers[:4], numbers[4:]
            standardized_features = self.standardize_features(np.array(features))
            return torch.tensor(standardized_features, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32)


In [36]:
filename = "/kaggle/input/harmonic/full_dataset.txt"
full_dataset = CustomDataset(filename, 2**20)

Lines found: 52051720
Calculating mean and std...
Mean: [ 6.6733383e+01 -5.4159999e-02  1.5670270e+00  3.1063519e+00], Std: [23.899603   41.23458     0.92115563  1.8126098 ]


In [37]:
train_size = int(0.0001 * len(full_dataset))
rest_size = len(full_dataset) - train_size
val_size = rest_size // 10000
test_size = rest_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(SEED))

In [38]:
train_size

5205

In [39]:
from torch.utils.data import DataLoader

BATCH_SIZE = 2 ** 22
train_shuffle = True
val_shuffle = False

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=train_shuffle)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=val_shuffle)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Models

In [40]:
import torch.nn as nn
import torch.nn.init as init

In [41]:
class Block(nn.Module):
    def __init__(self, in_layers, out_layers):
        super(Block, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_layers, out_layers),
            nn.ReLU(),
            nn.Dropout1d(p=0.1),
        )
        
    def forward(self, x):
        x = self.layers(x)
        return x

In [42]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.blocks = nn.ModuleList([
            Block(4, 256),
            Block(256, 256),
            Block(256, 256),
        ])
        self.bottleneck = nn.ModuleList([
            Block(256, 256),
            nn.Linear(256, 4),
        ])
        
        self.__init_weights()

    def __init_weights(self):
        layers = [self.blocks, self.bottleneck]
        # Initialize linear layers using Kaiming (He) uniform initialization
        for m in layers:
            for layer in m:
                self.__init_layer(layer)
                        
    def __init_layer(self, layer):
        if isinstance(layer, nn.Linear):
            init.kaiming_uniform_(layer.weight, mode='fan_in', nonlinearity='tanh')
            if layer.bias is not None:
                init.zeros_(layer.bias)

    def forward(self, x):
        x = self.blocks[0](x)
        for block in self.blocks[1:]:
            y = block(x)
            x = x + y
        for btl in self.bottleneck:
            x = btl(x)
        return x

In [60]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.blocks = nn.ModuleList([
            Block(4, 512),
            Block(512, 512),
        ])
        self.out = nn.Linear(512, 2)
        self.__init_weights()

    def __init_weights(self):
        layers = [self.blocks]
        # Initialize linear layers using Kaiming (He) uniform initialization
        for m in layers:
            for layer in m:
                self.__init_layer(layer)
        self.__init_layer(self.out)
                        
    def __init_layer(self, layer):
        if isinstance(layer, nn.Linear):
            init.kaiming_uniform_(layer.weight, mode='fan_in', nonlinearity='tanh')
            if layer.bias is not None:
                init.zeros_(layer.bias)

    def forward(self, x):
        x = self.blocks[0](x)
        for block in self.blocks[1:]:
            y = block(x)
            x = x + y
        x = self.out(x)
        return x

In [65]:
class RegressionAutoencoder(nn.Module):
    def __init__(self, encoder, decoder, freeze):
        super(RegressionAutoencoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.gradient(self.encoder, freeze)
        
    def gradient(self, model, freeze: bool):
        for parameter in model.parameters():
            parameter.requires_grad_(not freeze)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Methods

In [45]:
from torch.utils.tensorboard import SummaryWriter

def train(is_encoder, model, dataloader, optimizer, scheduler, loss_fn, epoch, writer, log_perc = 0.1):
    model.train()
    total_loss = 0
    total_diff = 0
    best_diff = 100

    logs_steps = max(int(log_perc * len(dataloader)), 1)
    start_step = epoch * len(dataloader)

    before_lr = optimizer.param_groups[0]["lr"]
    writer.add_scalar('Lr/Train', before_lr, epoch)
    for idx, (inputs, targets) in enumerate(tqdm(dataloader)):
        inputs, targets = inputs.to(DEVICE).to(torch.float32), targets.to(DEVICE).to(torch.float32)
        
        if is_encoder:
            targets = inputs
        
        optimizer.zero_grad()

        outputs = model(inputs)

        loss = loss_fn(outputs, targets)
        diff = torch.abs(outputs - targets).mean()

        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        total_diff += diff.item()
        
        if idx % logs_steps == 0:
            writer.add_scalar('Loss/Train', loss.item(), start_step + idx)
            writer.add_scalar('Absolute Difference/Train', diff.item(), start_step + idx)
            
            if SHOULD_PRINT:
                print(f"Loss/Train: {loss.item()}")
                print(f"Absolute Difference/Train: {diff.item()}")
        
    after_lr = optimizer.param_groups[0]["lr"]
    average_loss = total_loss / len(dataloader)
    average_diff = total_diff / len(dataloader)
    
    writer.add_scalar('Avg Loss/Train', average_loss, epoch)
    writer.add_scalar('Avg Absolute Difference/Train', average_diff, epoch)
    writer.add_scalar('Lr/Train', after_lr, epoch)
    
    if SHOULD_PRINT:
        print(f"Avg Loss/Train: {average_loss}")
        print(f"Avg Absolute Difference/Train: {average_diff}")
        print(f"Lr/Train: {after_lr}")

    # print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] Train Loss: {average_loss:.4f}, Train Diff: {average_diff:.15f}")

In [51]:
def validate(is_encoder, model, dataloader, loss_fn, epoch, writer):
    model.eval()
    total_loss = 0
    total_diff = 0

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(DEVICE).to(torch.float32), targets.to(DEVICE).to(torch.float32)
            
            if is_encoder:
                targets = inputs
            
            outputs = model(inputs)

            loss = loss_fn(outputs, targets)
            diff = torch.abs(outputs - targets).mean()
            
            total_loss += loss.item()
            total_diff += diff.item()

    average_loss = total_loss / len(dataloader)
    average_diff = total_diff / len(dataloader)

    if writer is not None:
        writer.add_scalar('Avg Loss/Val', average_loss, epoch)
        writer.add_scalar('Avg Absolute Difference/Val', average_diff, epoch)
    
    if SHOULD_PRINT:
        print(f"Avg Loss/Val: {average_loss}")
        print(f"Avg Absolute Difference/Val: {average_diff}")

    if epoch is not None:
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] Val Loss: {average_loss:.4f}, Val Diff: {average_diff:.15f}")
    else:
        print(f"Test Loss: {average_loss:.4f}, Test Diff: {average_diff:.15f}")
        
    return average_diff

# Encoder Training

In [52]:
LR = 1e-3
NUM_EPOCHS = 1
WEIGHT_DECAY = 0.99
WEIGHT_DECAY_L1 = 1e-4
MOMENTUM = 0.9

In [53]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

last_epoch = 0
encoder_model = Encoder()
loss_mse = nn.MSELoss()
optimizer = optim.SGD(encoder_model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY, nesterov=True)
scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, verbose=True)
if CONTINUE_MODEL:
    checkpoint = torch.load("encoder_checkpoint.pth")
    encoder_model.load_state_dict(checkpoint['encoder_model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    last_epoch = checkpoint['epoch']
    loss_mse = checkpoint['loss_mse']

Epoch 00000: adjusting learning rate of group 0 to 1.0000e-03.


In [54]:
def total_loss(outputs, targets):
    loss = loss_mse(outputs, targets)
#     loss_rmse = torch.sqrt(loss)
#     l1_reg = torch.tensor(0., requires_grad=True)

#     for name, param in encoder_model.named_parameters():
#         if 'weight' in name:
#             l1_reg = l1_reg + torch.linalg.norm(param, 1)

#     total_loss = (loss * 0.8 + loss_rmse * 0.2) + WEIGHT_DECAY_L1 * l1_reg
#     total_loss = (loss * 0.8 + loss_rmse * 0.2)
    return loss

In [55]:
import datetime

now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(f"tb_logs/encoder/{now}")

VALIDATION_STEPS = 6

encoder_model = encoder_model.to(DEVICE)
best_avg_diff = 1000

for idx, epoch in enumerate(range(last_epoch, NUM_EPOCHS)):
    train(True, encoder_model, train_dataloader, optimizer, scheduler, total_loss, epoch, writer, log_perc=0.2)

    if idx % VALIDATION_STEPS == 0:
        average_diff = validate(True, encoder_model, val_dataloader, total_loss, epoch, writer)
        if average_diff < best_avg_diff:
            torch.save({
                'epoch': epoch,
                'encoder_model_state_dict': encoder_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'loss_mse': loss_mse,
                }, "encoder_checkpoint.pth")

# Launch TensorBoard: `tensorboard --logdir=tb_logs`

100%|██████████| 1/1 [00:02<00:00,  2.19s/it]

Epoch 00001: adjusting learning rate of group 0 to 9.7553e-04.
Loss/Train: 1.6271324157714844
Absolute Difference/Train: 1.0536119937896729
Avg Loss/Train: 1.6271324157714844
Avg Absolute Difference/Train: 1.0536119937896729
Lr/Train: 0.0009755282581475768


Avg Loss/Val: 1.3871269226074219
Avg Absolute Difference/Val: 0.9900140762329102
Epoch [1/1] Val Loss: 1.3871, Val Diff: 0.990014076232910


In [56]:
#validate(True, encoder_model, test_dataloader, total_loss, epoch, None)

KeyboardInterrupt: 

# Regression Autoencoder Model

In [66]:
LR = 5e-4
NUM_EPOCHS = 1
WEIGHT_DECAY = 0.99
WEIGHT_DECAY_L1 = 1e-4
MOMENTUM = 0.9

In [67]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

last_epoch = 0
decoder = Decoder()
autoencoder_model = RegressionAutoencoder(encoder=encoder_model, decoder=decoder, freeze=True)
loss_mse = nn.MSELoss()
optimizer = optim.SGD(autoencoder_model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY, nesterov=True)
scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, verbose=True)
if CONTINUE_MODEL:
    checkpoint = torch.load("regressor_freezed_checkpoint.pth")
    autoencoder_model.load_state_dict(checkpoint['autoencoder_model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    last_epoch = checkpoint['epoch']
    loss_mse = checkpoint['loss_mse']

Epoch 00000: adjusting learning rate of group 0 to 5.0000e-04.


In [68]:
def total_loss(outputs, targets):
    loss = loss_mse(outputs, targets)
#     loss_rmse = torch.sqrt(loss)
#     l1_reg = torch.tensor(0., requires_grad=True)

#     for name, param in autoencoder_model.named_parameters():
#         if 'weight' in name:
#             l1_reg = l1_reg + torch.linalg.norm(param, 1)

#     total_loss = (loss * 0.8 + loss_rmse * 0.2) + WEIGHT_DECAY_L1 * l1_reg
#     total_loss = (loss * 0.8 + loss_rmse * 0.2)
    return loss

In [69]:
import datetime

now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(f"tb_logs/regressor/{now}")

VALIDATION_STEPS = 6

autoencoder_model = autoencoder_model.to(DEVICE)
best_avg_diff = 1000

for idx, epoch in enumerate(range(last_epoch, NUM_EPOCHS)):
    train(False, autoencoder_model, train_dataloader, optimizer, scheduler, total_loss, epoch, writer, log_perc=0.2)

    if idx % VALIDATION_STEPS == 0:
        average_diff = validate(False, autoencoder_model, val_dataloader, total_loss, epoch, writer)
        if average_diff < best_avg_diff:
            torch.save({
                'epoch': epoch,
                'autoencoder_model_state_dict': autoencoder_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'loss_mse': loss_mse,
                }, "regressor_freezed_checkpoint.pth")

# Launch TensorBoard: `tensorboard --logdir=tb_logs`

100%|██████████| 1/1 [00:01<00:00,  1.92s/it]

Epoch 00001: adjusting learning rate of group 0 to 4.8776e-04.
Loss/Train: 0.5163660645484924
Absolute Difference/Train: 0.5990405082702637
Avg Loss/Train: 0.5163660645484924
Avg Absolute Difference/Train: 0.5990405082702637
Lr/Train: 0.0004877641290737884


Avg Loss/Val: 0.3422410190105438
Avg Absolute Difference/Val: 0.49869897961616516
Epoch [1/1] Val Loss: 0.3422, Val Diff: 0.498698979616165


In [70]:
#validate(False, autoencoder_model, test_dataloader, total_loss, epoch, None)

# Unfreezed Regression Autoencoder Model

In [71]:
LR = 5e-4
NUM_EPOCHS = 1
WEIGHT_DECAY = 0.99
WEIGHT_DECAY_L1 = 1e-4
MOMENTUM = 0.9

In [75]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

last_epoch = 0
decoder = Decoder()
autoencoder_model = RegressionAutoencoder(encoder=encoder_model, decoder=decoder, freeze=False)
loss_mse = nn.MSELoss()
optimizer = optim.SGD(autoencoder_model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY, nesterov=True)
scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, verbose=True)
if CONTINUE_MODEL:
    checkpoint = torch.load("regressor_unfreezed_checkpoint.pth")
    autoencoder_model.load_state_dict(checkpoint['autoencoder_model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    last_epoch = checkpoint['epoch']
    loss_mse = checkpoint['loss_mse']

Epoch 00000: adjusting learning rate of group 0 to 5.0000e-04.


In [76]:
def total_loss(outputs, targets):
    loss = loss_mse(outputs, targets)
#     loss_rmse = torch.sqrt(loss)
#     l1_reg = torch.tensor(0., requires_grad=True)

#     for name, param in autoencoder_model.named_parameters():
#         if 'weight' in name:
#             l1_reg = l1_reg + torch.linalg.norm(param, 1)

#     total_loss = (loss * 0.8 + loss_rmse * 0.2) + WEIGHT_DECAY_L1 * l1_reg
#     total_loss = (loss * 0.8 + loss_rmse * 0.2)
    return loss

In [77]:
import datetime

now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(f"tb_logs/regressor/{now}")

VALIDATION_STEPS = 6

autoencoder_model = autoencoder_model.to(DEVICE)
best_avg_diff = 1000

for idx, epoch in enumerate(range(last_epoch, NUM_EPOCHS)):
    train(False, autoencoder_model, train_dataloader, optimizer, scheduler, total_loss, epoch, writer, log_perc=0.2)

    if idx % VALIDATION_STEPS == 0:
        average_diff = validate(False, autoencoder_model, val_dataloader, total_loss, epoch, writer)
        if average_diff < best_avg_diff:
            torch.save({
                'epoch': epoch,
                'autoencoder_model_state_dict': autoencoder_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'loss_mse': loss_mse,
                }, "regressor_unfreezed_checkpoint.pth")

# Launch TensorBoard: `tensorboard --logdir=tb_logs`

100%|██████████| 1/1 [00:02<00:00,  2.06s/it]

Epoch 00001: adjusting learning rate of group 0 to 4.8776e-04.
Loss/Train: 0.21327556669712067
Absolute Difference/Train: 0.33251211047172546
Avg Loss/Train: 0.21327556669712067
Avg Absolute Difference/Train: 0.33251211047172546
Lr/Train: 0.0004877641290737884


Avg Loss/Val: 0.12447430193424225
Avg Absolute Difference/Val: 0.2655174136161804
Epoch [1/1] Val Loss: 0.1245, Val Diff: 0.265517413616180


In [ ]:
#validate(False, autoencoder_model, test_dataloader, total_loss, epoch, None)